Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [1]:
from random import random, seed
from itertools import product
import numpy as np

from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [2]:
UNIVERSE_SIZE = 100_000
NUM_SETS = 10_000
DENSITY = 0.2

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [3]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)

## Helper Functions

In [4]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

## Have Fun!

In [5]:
# A dumb solution of "all" sets
solution = np.full(NUM_SETS, True)
valid(solution), cost(solution)

(np.True_, np.float64(538441135.3204354))

In [6]:
# A random solution with random 50% of the sets
solution = rng.random(NUM_SETS) < .5
valid(solution), cost(solution)

(np.True_, np.float64(268535316.02232045))

## Greedy Solution

In [7]:
solution = []
# covered elements in a set (starting with all elements as uncovered)
covered_elements = np.zeros(UNIVERSE_SIZE, dtype=bool)

# keep iterating until a valid solution is found
while not valid(solution):
    # to find the best set to take, we have to take the set with the higher ratio
    best_ratio = -1
    best_set = None
    max_discovered = 0
    
    for i, set in enumerate(SETS):
        # if the set i is already in the solution skip
        if i in solution:
            continue
        # finding the elements that are covered by the set i
        new_covered = np.logical_and(set, ~covered_elements)
        # number of true elements in the set
        num_discovered = np.sum(new_covered)
        
        if num_discovered >= max_discovered:
            # computing the ratio, identifying how much is important to take this set
            ratio = num_discovered / COSTS[i]
            if ratio > best_ratio:
                best_ratio = ratio
                best_set = i
                max_discovered = num_discovered

    solution.append(best_set)
    # updating the covered_elements with the new discovered elements by the best set
    covered_elements = np.logical_or(covered_elements, SETS[best_set])
    
    
print(valid(solution), cost(solution))

True 1671704.2807142711


## Utility Functions for Hill Climbing

In [8]:
# tweak function changing only one element in the solution
def single_tweak(solution):
    new_solution = solution.copy()
    index = rng.integers(0, NUM_SETS)
    new_solution[index] = not new_solution[index]
    return new_solution

# tweak function changing multiple elements in the solution
def multi_tweak(solution):
    new_solution = solution.copy()
    mask = rng.random(NUM_SETS) < .01
    new_solution = np.logical_xor(new_solution, mask)
    return new_solution

# function to evaluate the fitness of a solution
def fitness(solution):
    return (valid(solution), -cost(solution))

## Hill Climbing

#### Hill Climbing with single tweak

In [9]:
best_solution = np.full(NUM_SETS, True)
num_iters = 100

for step in range(num_iters):
    solution = rng.random(NUM_SETS) < .5
    
    while not valid(solution):
        solution = single_tweak(solution)
    
    if cost(solution) < cost(best_solution):
        best_solution = solution.copy()
        
print(valid(best_solution), cost(best_solution))

True 260755873.29564267


#### Hill Climbing with multiple tweak

In [10]:
best_solution = np.full(NUM_SETS, True)
num_iters = 100

for step in range(num_iters):
    solution = rng.random(NUM_SETS) < .5
    
    while not valid(solution):
        solution = multi_tweak(solution)
    
    if cost(solution) < cost(best_solution):
        best_solution = solution.copy()
        
print(valid(best_solution), cost(best_solution))

True 264761689.34614763


#### Conclusion looking the results

Analyzing the results, we can conclude that with the greedy algorithm we can obtain better performance, in term of validity and cost of the solution.